In [1]:
import traci

In [4]:
tls_ids = ['tls_159', 'tls_160', 'tls_161']

try:
    traci.close()
except:
    pass

traci.start(['sumo', "-c", "models/20230718_sumo_ma/osm.sumocfg", "--time-to-teleport", "300"])

tls_controlled_edges = {id : set(traci.lane.getEdgeID(lane) for lane in traci.trafficlight.getControlledLanes(id)) for id in tls_ids}


for i in range(10):
    traci.simulationStep()
    n_veh_to_pass_tls = {id: 0 for id in tls_ids}  # key: tls id; value: number of vehicles that will pass in future / is currently passing the tls
    n_veh_active = 0

    for veh in traci.vehicle.getIDList():
        route_edges = traci.vehicle.getRoute(veh)

        # If the route only contains 2 elements (start and end edge), the actual vehicle route is not yet computed
        if len(route_edges) >2:
            n_veh_active += 1
            route_edges_set = set(route_edges)
            for tls_id, controlled_edges in tls_controlled_edges.items():
                intersect = controlled_edges.intersection(route_edges_set)  # edges which the vehicle will pass (or has passed) that are controlled by the tls

                if len(intersect)>=1:   # TODO: check what to do if len>1 (i.e. vehicle will pass the tls multiple times)
                    tls_edge = intersect.pop()
                    veh_lane = traci.vehicle.getLaneID(veh)
                    if veh_lane[0] == ":":
                        veh_lane = traci.lane.getLinks(veh_lane)[0][0]  # if lane belongs to internal edge, select the following lane

                    veh_edge = traci.lane.getEdgeID(veh_lane)
                    if route_edges.index(veh_edge) <= route_edges.index(tls_edge):  # vehicle did not already pass the tls
                        #  TODO: here the closeness of the vehicle to the tls could be an additional requirement,
                        #  i.e. the vehicle_edge index not only has to be not larger than the tls_edge index but their difference
                        #  also has to be smaller than a certain amount
                        n_veh_to_pass_tls[tls_id] += 1

    print(n_veh_to_pass_tls, n_veh_active)

traci.close()

{'tls_159': 0, 'tls_160': 0, 'tls_161': 0} 4
{'tls_159': 1, 'tls_160': 1, 'tls_161': 1} 5
{'tls_159': 1, 'tls_160': 1, 'tls_161': 1} 6
{'tls_159': 3, 'tls_160': 3, 'tls_161': 2} 8
{'tls_159': 4, 'tls_160': 4, 'tls_161': 2} 10
{'tls_159': 4, 'tls_160': 4, 'tls_161': 2} 11
{'tls_159': 5, 'tls_160': 5, 'tls_161': 2} 13
{'tls_159': 5, 'tls_160': 5, 'tls_161': 2} 14
{'tls_159': 5, 'tls_160': 5, 'tls_161': 2} 16
{'tls_159': 6, 'tls_160': 6, 'tls_161': 2} 18
